In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('final_data.csv')

In [13]:
df.columns

Index(['Unnamed: 0', 'area', 'rank_gs', 'title', 'rate_gs', 'rank_pc',
       'rate_pc', 'rank_ss', 'rate_ss', 'agg_rate', 'agg_rank', 'rank',
       'pdf_link', 'abstract', 'file_name', 'citationCount', 'referenceCount',
       'influentialCitationCount'],
      dtype='object')

In [14]:
df.drop(['area', 'rank_gs', 'title',  'rate_gs', 'rank_pc',
       'rate_pc', 'rank_ss', 'rate_ss', 'agg_rate', 'agg_rank', 'rank',
       'pdf_link','file_name', 'citationCount', 'referenceCount',
       'influentialCitationCount'], axis = 1, inplace = True)

In [18]:
df.columns = ['id', 'contents']

In [19]:
df

,id,contents
0,0,"With the advent of deep neural networks, learn..."
1,1,Supervised 3D reconstruction has witnessed a s...
2,2,Volumetric models have become a popular repres...
3,3,Recovering the 3D representation of an object ...
4,4,One challenge that remains open in 3D deep lea...
...,...,...
8250,9095,Abstract Limited by the existed imagery hardwa...
8251,9096,Counting molecules in complexes is challenging...
8252,9098,Single image super-resolution (SR) is an ill-p...
8253,9099,We study sparse spikes super-resolution over t...


In [21]:
with open('documents/documents.json','w') as f:
    f.write(df.to_json(orient='records', lines=True))

In [23]:
! python -m pyserini.index -collection JsonCollection \
                          -generator DefaultLuceneDocumentGenerator \
                           -threads 1 \
                           -input documents/ \
                           -index indexes/ \
                           -storePositions -storeDocvectors -storeRaw    

2021-11-05 17:39:01,543 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Setting log level to INFO
2021-11-05 17:39:01,543 INFO  [main] index.IndexCollection (IndexCollection.java:646) - Starting indexer...
2021-11-05 17:39:01,543 INFO  [main] index.IndexCollection (IndexCollection.java:647) - ============ Loading Parameters ============
2021-11-05 17:39:01,543 INFO  [main] index.IndexCollection (IndexCollection.java:648) - DocumentCollection path: documents/
2021-11-05 17:39:01,543 INFO  [main] index.IndexCollection (IndexCollection.java:649) - CollectionClass: JsonCollection
2021-11-05 17:39:01,543 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Generator: DefaultLuceneDocumentGenerator
2021-11-05 17:39:01,543 INFO  [main] index.IndexCollection (IndexCollection.java:651) - Threads: 1
2021-11-05 17:39:01,543 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Stemmer: porter
2021-11-05 17:39:01,543 INFO  [main] index.IndexCollection (IndexCol

In [24]:
from pyserini.index import IndexReader
index_reader = IndexReader('indexes/')

In [119]:
query = 'I would like to reconstruct 3D models from images. '
docids = list(df.id.astype('str'))

result = pd.DataFrame(columns=['docid', 'score'])
counter = 0

result


,docid,score


In [120]:
for i in range(0, len(docids)):
    score = index_reader.compute_query_document_score(docids[i], query)
    result.loc[counter] = [docids[i], score]
    counter+=1

In [121]:
result.sort_values(['score'], ascending = False).docid[0:20]

2          2
4726    5210
7069    7779
106      106
135      136
346      355
373      383
391      403
171      172
1916    2061
1167    1237
2787    3045
224      228
69        69
7063    7773
1919    2064
3298    3606
29        29
87        87
372      382
Name: docid, dtype: object

In [122]:
L = list(result.sort_values(['score'], ascending = False).docid[0:20])

In [123]:
for i in range(len(L)):
    L[i] = int(L[i])

In [131]:
query_result = list(df[df.id.isin(L)].contents)

In [148]:
df_test = pd.read_csv('test_set.csv')
df_test.columns = ['Query', 'Unnamed: 1', 'Unnamed: 2', 'title', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'abstract']
all_list = list(df_test[df_test.Query == query].abstract)
rank_result = []
for i in range(20):
    if query_result[i] in all_list:
        ind = all_list.index(query_result[i])
        if ind < 20:
            rank_result.append(5)
        elif 20 <= ind < 40:
            rank_result.append(4)
        elif 40 <= ind < 60:
            rank_result.append(3)
        elif 60 <= ind < 80:
            rank_result.append(2)
        else:
            rank_result.append(1)
    else:
        rank_result.append(0)

In [149]:
rank_result

[0, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [171]:
import math
DCG = 0
for i in range(len(rank_result)):
    if i == 0:
        DCG += rank_result[i]
    else:
        DCG += (rank_result[i]) / math.log(1 + i, 2)
IDCG = 0
s_list = sorted(rank_result, reverse = True)
for i in range(len(s_list)):
    if i == 0:
        IDCG += s_list[i]
    else:
        IDCG += (s_list[i]) / math.log(1 + i, 2)
print(DCG / IDCG)

0.9


In [200]:
def calculate_query_NDCG(df, query):
    from pyserini.index import IndexReader
    index_reader = IndexReader('indexes/')
    docids = list(df.id.astype('str'))
    result = pd.DataFrame(columns=['docid', 'score'])
    counter = 0
    for i in range(0, len(docids)):
        score = index_reader.compute_query_document_score(docids[i], query)
        result.loc[counter] = [docids[i], score]
        counter+=1
    L = list(result.sort_values(['score'], ascending = False).docid[0:20])
    for i in range(len(L)):
        L[i] = int(L[i])
    query_result = list(df[df.id.isin(L)].contents)
    df_test = pd.read_csv('test_set.csv')
    df_test.columns = ['Query', 'Unnamed: 1', 'Unnamed: 2', 'title', 'Unnamed: 4',
           'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
           'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'abstract']
    all_list = list(df_test[df_test.Query == query].abstract)
    rank_result = []
    for i in range(20):
        if query_result[i] in all_list:
            ind = all_list.index(query_result[i])
            if ind < 20:
                rank_result.append(5)
            elif 20 <= ind < 40:
                rank_result.append(4)
            elif 40 <= ind < 60:
                rank_result.append(3)
            elif 60 <= ind < 80:
                rank_result.append(2)
            else:
                rank_result.append(1)
        else:
            rank_result.append(0)
    print('Relevance: ',rank_result)
    import math
    DCG = 0
    for i in range(len(rank_result)):
        if i == 0:
            DCG += rank_result[i]
        else:
            DCG += (rank_result[i]) / math.log(1 + i, 2)
    IDCG = 0
    s_list = sorted(rank_result, reverse = True)
    for i in range(len(s_list)):
        if i == 0:
            IDCG += s_list[i]
        else:
            IDCG += (s_list[i]) / math.log(1 + i, 2)
    if IDCG == 0:
        return 1
    return DCG / IDCG

In [190]:
df_test = pd.read_csv('test_set.csv')

In [191]:
df_test = pd.read_csv('test_set.csv')
df_test.columns = ['Query', 'Unnamed: 1', 'Unnamed: 2', 'title', 'Unnamed: 4',
           'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
           'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'abstract']

In [201]:
counter = 1
total_NDCG = 0
for query in list(df_test.Query.drop_duplicates()):
    print('Query', counter)
    counter += 1
    val = calculate_query_NDCG(df, query)
    print(f'NDCG: {val}')
    total_NDCG += val

Query 1
Relevance:  [0, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NDCG: 0.9
Query 2
Relevance:  [4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NDCG: 1.0
Query 3
Relevance:  [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NDCG: 1.0
Query 4
Relevance:  [4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NDCG: 1.0
Query 5
Relevance:  [0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NDCG: 0.4087646826539674
Query 6
Relevance:  [5, 4, 4, 4, 4, 4, 4, 3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NDCG: 1.0
Query 7
Relevance:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NDCG: 1
Query 8
Relevance:  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NDCG: 0.5
Query 9
Relevance:  [5, 5, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0]
NDCG: 0.9242474420301763
Query 10
Relevance:  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NDCG: 0.3154648767857287
Query 11
Relevance:  [0, 0, 0, 0, 0, 0, 0, 0, 

In [202]:
total_NDCG / 30

0.6571098005058454